In [9]:
import os
import json
import uuid
import time
import asyncio
from confluent_kafka.serialization import SerializationContext, MessageField
from prometheus_client import start_http_server
from confluent_kafka import KafkaException
from src.metrics import *
from src.logging import log
from src.rpc_provider import Web3AsyncRouter, AsyncRpcClient, AsyncRpcScheduler, RpcPool, RpcErrorResult, RpcTaskMeta
from src.state import load_last_state
from src.kafka_utils import init_producer, get_serializers, delivery_report
from src.web3_utils import current_utctime, to_json_safe
# from src.commit_timer import CommitTimer
# from src.batch_executor import BatchContext, ParallelBatchExecutor
from src.safe_latest import SafeLatestBlockProvider

# -----------------------------
# Environment Variables
# -----------------------------
RUN_ID = os.getenv("RUN_ID", str(uuid.uuid4()))
POLL_INTERVAL = float(os.getenv("POLL_INTERVAL", "1")) # can be decimals
CHAIN = os.getenv("CHAIN", "bsc").lower() # bsc, eth, base ... from blockchain-rpc-config.yaml
RESUME_FROM_LAST = os.getenv("RESUME_FROM_LAST", "True").lower() in ("1", "true", "yes")

# -----------------------------
# Job Name & Kafka IDs
# -----------------------------
if RESUME_FROM_LAST:
    JOB_NAME = f"{CHAIN}_backfill_resume"        # 固定名，Kafka checkpoint 能被复用
else:
    JOB_NAME = f"{CHAIN}_realtime_{current_utctime()}"  # 每次唯一，从最新block开始

# -----------------------------
# RPC Config
# -----------------------------
RPC_CONFIG_PATH = "/etc/ingestion/rpc_providers.json"
RPC_MAX_TIMEOUT = int(os.getenv("RPC_MAX_TIMEOUT", "10"))
RPC_MAX_SUBMIT = int(os.getenv("RPC_MAX_SUBMIT", "15")) # how many blocks for every commit to Kafka
RPC_MAX_INFLIGHT = int(os.getenv("RPC_MAX_INFLIGHT", "10"))  # 并发数量

# -----------------------------
# Log fetching Config
# -----------------------------
RANGE_SIZE = int(os.getenv("RANGE_SIZE", "5")) # how many blocks of range to fetch for logs
BATCH_TX_SIZE = int(os.getenv("BATCH_TX_SIZE", "5"))  # Max 10 logs transaction per batch within a single block
SLIDING_SIZE = int(os.getenv("SLIDING_SIZE", "20"))

# -----------------------------
# Kafka Config
# -----------------------------
TRANSACTIONAL_ID = f"blockchain.ingestion.{CHAIN}.{current_utctime()}" # TRANSACTIONAL_ID每次不一样，EOS由Compact State Topic实现
KAFKA_BROKER = "redpanda.kafka.svc:9092"
SCHEMA_REGISTRY_URL = "http://redpanda.kafka.svc:8081"
BLOCKS_TOPIC = f"blockchain.logs.{CHAIN}"
STATE_TOPIC = f"blockchain.state.{CHAIN}"

# -----------------------------
# RPC Initilization
# -----------------------------
rpc_configs = json.load(open(RPC_CONFIG_PATH))
rpc_pool = RpcPool.from_config(rpc_configs, CHAIN)

# -----------------------------
# Kafka Producer initialization
# -----------------------------
blocks_value_serializer, state_value_serializer = get_serializers(SCHEMA_REGISTRY_URL, BLOCKS_TOPIC, STATE_TOPIC)
producer = init_producer(TRANSACTIONAL_ID, KAFKA_BROKER)

{"ts": "2026-01-23T09:58:09.857Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "quiknode", "key_envs": ["QUIKNODE_KEY"], "weight": 8}
{"ts": "2026-01-23T09:58:09.858Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "infura", "key_envs": ["INFURA_KEY_A", "INFURA_KEY_B", "INFURA_KEY_C"], "weight": 8}
{"ts": "2026-01-23T09:58:09.859Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "blockpi", "key_envs": ["BLOCKPI_BSC_KEY_A", "BLOCKPI_BSC_KEY_B", "BLOCKPI_BSC_KEY_C"], "weight": 8}
{"ts": "2026-01-23T09:58:09.859Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "ankr", "key_envs": ["ANKR_KEY_A", "ANKR_KEY_B", "ANKR_KEY_C"], "weight": 6}
{"ts": "2026-01-23T09:58:09.860Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "public-blxrbdn", "key_envs": ["public"], "weight": 2}
{"

In [3]:
# Block Range 任务
from dataclasses import dataclass

@dataclass
class BlockRangeTask:
    range_id: int
    start_block: int
    end_block: int
    
# Range 执行结果（RPC → Kafka 的单位）
@dataclass
class RangeResult:
    range_id: int
    start_block: int
    end_block: int
    logs: list
    rpc: str
    key_env: str
    task_id: int


# Range Planner（顺序是从这里开始的）
def plan_block_ranges(start_block: int, end_block: int, range_size: int):
    range_id = 0
    b = start_block

    while b <= end_block:
        yield BlockRangeTask(
            range_id=range_id,
            start_block=b,
            end_block=min(b + range_size - 1, end_block),
        )
        b += range_size
        range_id += 1

# OrderedResultBuffer（保证顺序提交）- ingestion 的灵魂组件
class OrderedResultBuffer:
    def __init__(self):
        self._buffer = {}
        self._next_range_id = 0

    def add(self, result: RangeResult):
        self._buffer[result.range_id] = result

    def pop_ready(self):
        ready = []
        while self._next_range_id in self._buffer:
            ready.append(self._buffer.pop(self._next_range_id))
            self._next_range_id += 1
        return ready

In [4]:
class RangeCursor:
    def __init__(self, start_block: int, range_size: int):
        self.next_block = start_block
        self.range_size = range_size
        self.range_id = 0

    def next_range(self, max_block: int) -> BlockRangeTask | None:
        if self.next_block > max_block:
            return None

        start = self.next_block
        end = min(start + self.range_size - 1, max_block)

        r = BlockRangeTask(
            range_id=self.range_id,
            start_block=start,
            end_block=end,
        )

        self.next_block = end + 1
        self.range_id += 1
        return r

In [5]:
class LatestBlockTracker:
    def __init__(self, router, refresh_interval=2.0):
        self.router = router
        self.refresh_interval = refresh_interval
        self._latest = None
        self._last_refresh = 0.0
        self._lock = asyncio.Lock()

    async def get(self) -> int:
        async with self._lock:
            now = time.monotonic()
            if (
                self._latest is None
                or now - self._last_refresh >= self.refresh_interval
            ):
                self._latest = await self.router.get_latest_block()
                self._last_refresh = now
            return self._latest


In [6]:
async def submit_range(
    scheduler: AsyncRpcScheduler,
    r: BlockRangeTask,
):
    params = [{
        "fromBlock": hex(r.start_block),
        "toBlock": hex(r.end_block),
    }]

    return asyncio.create_task(
        scheduler.submit(
            "eth_getLogs",
            params,
            meta={
                "range_id": r.range_id,
                "start_block": r.start_block,
                "end_block": r.end_block,
            },
        )
    )


In [11]:
async def stream_ingest_logs(
    *,
    start_block: int,
    range_size: int,
    rpc_pool,
    producer,
    max_inflight_ranges: int = 20,
):
    # -----------------------------
    # RPC infra
    # -----------------------------
    client = AsyncRpcClient(timeout=RPC_MAX_TIMEOUT)
    router = Web3AsyncRouter(rpc_pool, client)

    scheduler = AsyncRpcScheduler(
        router=router,
        max_workers=1,
        max_inflight=RPC_MAX_INFLIGHT,
        max_queue=RPC_MAX_INFLIGHT * 3,
    )

    # -----------------------------
    # Control plane
    # -----------------------------
    latest_tracker = LatestBlockTracker(router, refresh_interval=2.0)
    cursor = RangeCursor(start_block, range_size)

    # -----------------------------
    # Ordering
    # -----------------------------
    ordered_buffer = OrderedResultBuffer()

    # -----------------------------
    # Inflight window
    # -----------------------------
    inflight: set[asyncio.Task] = set()

    # -----------------------------
    # 预填满滑动窗口
    # -----------------------------
    latest_block = await latest_tracker.get()

    while len(inflight) < max_inflight_ranges:
        r = cursor.next_range(latest_block)
        if not r:
            break
        inflight.add(await submit_range(scheduler, r))

    # -----------------------------
    # 主 streaming loop
    # -----------------------------
    while inflight:
        done, _ = await asyncio.wait(
            inflight,
            return_when=asyncio.FIRST_COMPLETED,
        )

        for task in done:
            inflight.remove(task)
            result = await task

            # ---------- RPC error ----------
            if isinstance(result, RpcErrorResult):
                log.error(
                    "rpc_range_failed",
                    extra={
                        "range_id": result.meta.extra.get("range_id"),
                        "error": type(result.error).__name__,
                    },
                )
                raise RuntimeError("range rpc failed")

            logs, rpc, key_env, trace, wid, meta = result

            rr = RangeResult(
                range_id=meta.extra["range_id"],
                start_block=meta.extra["start_block"],
                end_block=meta.extra["end_block"],
                logs=logs or [],
                rpc=rpc,
                key_env=key_env,
                task_id=meta.task_id,
            )

            ordered_buffer.add(rr)

            # ---------- 顺序消费 ----------
            ready_ranges = ordered_buffer.pop_ready()

            for rr in ready_ranges:
                # ===== Kafka EOS 写入 =====
                producer.begin_transaction()

                logs_by_block = {}
                for log_item in to_json_safe(rr.logs):
                    bn = log_item.get("blockNumber")
                    if bn is None:
                        continue
                    if isinstance(bn, str):
                        bn = int(bn, 16)
                    logs_by_block.setdefault(bn, []).append(log_item)

                for bn, txs in logs_by_block.items():
                    for idx, tx in enumerate(txs):
                        producer.produce(
                            topic=BLOCKS_TOPIC,
                            key=f"{bn}-{idx}",
                            value=blocks_value_serializer(
                                {
                                    "block_height": bn,
                                    "job_name": JOB_NAME,
                                    "run_id": RUN_ID,
                                    "inserted_at": current_utctime(),
                                    "raw": json.dumps(tx),
                                    "tx_index": idx,
                                },
                                SerializationContext(
                                    BLOCKS_TOPIC, MessageField.VALUE
                                ),
                            ),
                        )
                    producer.poll(0)

                # checkpoint
                producer.produce(
                    STATE_TOPIC,
                    key=JOB_NAME,
                    value=state_value_serializer(
                        {
                            "job_name": JOB_NAME,
                            "run_id": f"{RUN_ID}-task-{rr.task_id}",
                            "range": {
                                "start": rr.start_block,
                                "end": rr.end_block,
                            },
                            "checkpoint": rr.end_block,
                            "status": "running",
                            "inserted_at": current_utctime(),
                        },
                        SerializationContext(
                            STATE_TOPIC, MessageField.VALUE
                        ),
                    ),
                )
                producer.poll(0)
                producer.commit_transaction()

                log.info(
                    "range_committed",
                    extra={
                        "range_id": rr.range_id,
                        "start": rr.start_block,
                        "end": rr.end_block,
                        "rpc": rr.rpc,
                        "log_count": len(rr.logs),
                    },
                )

            # ---------- 下游释放 → 上游补位 ----------
            latest_block = await latest_tracker.get()

            while len(inflight) < max_inflight_ranges:
                r = cursor.next_range(latest_block)
                if not r:
                    break
                inflight.add(await submit_range(scheduler, r))

    # -----------------------------
    # shutdown
    # -----------------------------
    await scheduler.close()

In [ ]:
# Entrypoint
if __name__ == "__main__":
    # Prometheus metrics endpoint
    # start_http_server(8000)

    last_state = load_last_state(
        job_name=JOB_NAME,
        kafka_broker=KAFKA_BROKER,
        state_topic=STATE_TOPIC,
        schema_registry_url=SCHEMA_REGISTRY_URL,
    )

    last_block = last_state["checkpoint"]

    start_block = last_block + 1

    await stream_ingest_logs(start_block=start_block,
        range_size=RANGE_SIZE,
        rpc_pool=rpc_pool,
        producer=producer)

{"ts": "2026-01-23T10:00:19.920Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatcher_started", "worker": 0}
{"ts": "2026-01-23T10:00:20.194Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 1, "method": "eth_getLogs", "queue_wait_ms": 0.09}
{"ts": "2026-01-23T10:00:20.195Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 2, "method": "eth_getLogs", "queue_wait_ms": 0.93}
{"ts": "2026-01-23T10:00:20.195Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 3, "method": "eth_getLogs", "queue_wait_ms": 1.4}
{"ts": "2026-01-23T10:00:20.195Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 4, "method": "eth_getLogs", "queue_wait_ms": 1.81}
{"ts": "2026-01-23T10:00:20.196Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 5, "method": "eth_getLogs", "queue_wait_ms": 2.61}
{"ts": "2026-01-23T10:00:20.197Z", "level": "INFO"

CancelledError: 

{"ts": "2026-01-23T10:01:25.513Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_done", "task_id": 370, "rpc": "nownodes", "key_env": "NOWNODES_KEY_A", "latency_ms": 3074.73}
{"ts": "2026-01-23T10:01:25.514Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_start", "task_id": 387, "method": "eth_getLogs"}
{"ts": "2026-01-23T10:01:25.516Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_done", "task_id": 377, "rpc": "ankr", "key_env": "ANKR_KEY_B", "latency_ms": 2113.4}
{"ts": "2026-01-23T10:01:25.516Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_start", "task_id": 388, "method": "eth_getLogs"}
{"ts": "2026-01-23T10:01:25.518Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_done", "task_id": 383, "rpc": "drpc", "key_env": "DRPC_API_KEY_B", "latency_ms": 1192.56}
{"ts": "2026-01-23T10:01:25.519Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_call_start", "task_id": 389, "method": "eth_getLogs"}
{"